In [1]:
import pymysql
import sqlalchemy as alch
from getpass import getpass
import pandas as pd

In [48]:
coordenadas = pd.read_csv('/Users/haddock_ana/Ironhack/final_project/Data/wine_analysis/Data/coordenadas_regions_1906.csv')

In [49]:
wines = pd.read_csv('/Users/haddock_ana/Ironhack/final_project/Data/wine_analysis/Data/XWines_Full_100K_wines.csv')

In [50]:
clima = pd.read_csv('/Users/haddock_ana/Ironhack/final_project/Data/wine_analysis/Data/climate_df_1906_no_nulls.csv')

In [51]:
clima.head()

,Unnamed: 0,RegionName,time,season,tmin,tmax,prcp,snow,pres,tsun,Latitude,Longitude,Country
0,0,Santiago,2018,3.0,11.396629,17.900000,0.0,0.0,1017.751685,256.908696,41.0675,-83.798075,Chile
1,1,Santiago,2018,2.0,16.490625,24.810938,0.0,0.0,1017.285938,292.867467,41.0675,-83.798075,Chile
2,2,Santiago,2018,0.0,9.054545,15.118182,0.0,0.0,1029.263636,186.526276,41.0675,-83.798075,Chile
3,3,Santiago,2019,3.0,10.883146,16.330337,0.0,0.0,1016.901124,128.487640,41.0675,-83.798075,Chile
4,4,Santiago,2019,1.0,11.098913,19.128261,0.0,0.0,1016.690217,377.223913,41.0675,-83.798075,Chile


In [ ]:
ratings = pd.read_csv('/Users/haddock_ana/Ironhack/final_project/Data/wine_analysis/Data/XWines_Full_21M_ratings.csv')

In [53]:
def assign_region_id(row, wines_df):
    region_name = row['RegionName']
    region_id = wines_df[wines_df['RegionName'] == region_name]['RegionID']
    if not region_id.empty:
        return region_id.values[0]
    return None

# Asignar RegionID a coordenadas
coordenadas['RegionID'] = coordenadas.apply(lambda row: assign_region_id(row, wines), axis=1)

# Asignar RegionID a clima
clima['RegionID'] = clima.apply(lambda row: assign_region_id(row, wines), axis=1)


In [57]:
clima.to_csv('climate_df_1906_no_nulls.csv')

In [58]:
clima.drop(columns=['Unnamed: 0', 'Country', 'Longitude', 'Latitude', 'RegionName' ], inplace= True)

In [62]:
coordenadas.drop(columns=['Unnamed: 0'],inplace=True)

In [61]:
wines.drop(columns=['Harmonize', 'Website', 'Vintages'], inplace=True)

In [63]:
# Limpiar la columna Elaborate
wines['Elaborate'] = wines['Elaborate'].apply(lambda x: x.split('Assemblage/')[-1] if 'Assemblage/' in x else x)

In [64]:
wines.head()

,WineID,WineName,Type,Elaborate,Grapes,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare
3,100004,Virtus Moscato,White,Varietal/100%,['Muscat/Moscato'],12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10003,Monte Paschoal
4,100005,Maison de Ville Cabernet-Merlot,Red,Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora


In [102]:
wines.to_csv('wines_final_data.csv')

In [15]:
password = getpass('Introduce tu contraseña')

In [101]:
# Connect to MySQL
conn = pymysql.connect(
    host='localhost', # select user() en my sql para ver el host
    user='root',
    password=password,
    database='bank' # esto es para tener algo en comun, una vez dentro se pueden crear otras db
)
# Create a cursor object
cursor = conn.cursor() # esto permite interaccionar entre las dos vias, ejecuta directamente como si estuvieras en sql
# Create database
cursor.execute("CREATE DATABASE IF NOT EXISTS Wines")
# Select the database
cursor.execute("USE Wines")

KeyboardInterrupt: 

In [83]:
# Crear la tabla Coordenadas con RegionID
create_table_coordenadas = """
CREATE TABLE IF NOT EXISTS Coordenadas (
    RegionID INT,
    Country VARCHAR(255),
    RegionName VARCHAR(255) NOT NULL,
    Latitude DOUBLE,
    Longitude DOUBLE,
    PRIMARY KEY (RegionID)
);
"""
cursor.execute(create_table_coordenadas)

# Crear la tabla Clima con RegionID
create_table_clima = """
CREATE TABLE IF NOT EXISTS Clima (
    ClimaID INT AUTO_INCREMENT,
    RegionID INT,
    time YEAR,
    season INT,
    tmin DOUBLE,
    tmax DOUBLE,
    prcp DOUBLE,
    snow DOUBLE,
    pres DOUBLE,
    tsun DOUBLE,
    Latitude DOUBLE,
    Longitude DOUBLE,
    PRIMARY KEY (ClimaID),
    FOREIGN KEY (RegionID) REFERENCES Coordenadas(RegionID)
);
"""
cursor.execute(create_table_clima)

# Crear la tabla Wines
create_table_wines = """
CREATE TABLE IF NOT EXISTS Wines (
    WineID INT,
    WineName VARCHAR(255),
    Type VARCHAR(50),
    Elaborate VARCHAR(255),
    Grapes VARCHAR(255),
    ABV FLOAT,
    Body VARCHAR(255),
    Acidity VARCHAR(255),
    Code VARCHAR(50),
    Country VARCHAR(255),
    RegionID INT,
    WineryID INT,
    WineryName VARCHAR(255),
    PRIMARY KEY (WineID),
    FOREIGN KEY (RegionID) REFERENCES Coordenadas(RegionID)
);
"""
cursor.execute(create_table_wines)

# Crear la tabla Ratings
create_table_ratings = """
CREATE TABLE IF NOT EXISTS Ratings (
    RatingID INT AUTO_INCREMENT,
    UserID INT,
    WineID INT,
    Vintage YEAR,
    Rating FLOAT,
    Date DATE,
    PRIMARY KEY (RatingID),
    FOREIGN KEY (WineID) REFERENCES Wines(WineID)
);
"""
cursor.execute(create_table_ratings)

0

In [86]:
# Función para insertar datos en las tablas
def insert_data(table_name, dataframe):
    placeholders = ', '.join(['%s'] * len(dataframe.columns))
    columns = ', '.join(dataframe.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    for row in dataframe.itertuples(index=False, name=None):
        cursor.execute(sql, row)
    conn.commit()

# Preparar los DataFrames para la inserción
coordenadas_df = coordenadas[['RegionID', 'Country', 'RegionName', 'Latitude', 'Longitude']]
clima_df = clima[['RegionID', 'time', 'season', 'tmin', 'tmax', 'prcp', 'snow', 'pres', 'tsun']]
wines_df = wines[['WineID', 'WineName', 'Type', 'Elaborate', 'Grapes', 'ABV', 'Body', 'Acidity', 'Code', 'Country', 'RegionID', 'WineryID', 'WineryName']]
ratings_df = ratings[['RatingID', 'UserID', 'WineID', 'Vintage', 'Rating', 'Date']]

# Insertar los datos en las tablas
insert_data('Coordenadas', coordenadas_df)
insert_data('Clima', clima_df)
insert_data('Wines', wines_df)
insert_data('Ratings', ratings_df)

# Cerrar la conexión
cursor.close()
conn.close()

IntegrityError: (1062, "Duplicate entry 'Pyrenees' for key 'coordenadas.RegionName'")

In [84]:
# Verificar duplicados en coordenadas
duplicados_coordenadas = coordenadas[coordenadas.duplicated(subset=['RegionID'], keep=False)]
print("Duplicados en coordenadas:")
print(duplicados_coordenadas)

Duplicados en coordenadas:
Empty DataFrame
Columns: [Country, RegionName, Latitude, Longitude, RegionID]
Index: []


In [97]:
coordenadas[coordenadas['RegionName']=='Pyrenees']

,Country,RegionName,Latitude,Longitude,RegionID
37139,Australia,Pyrenees,43.16188,0.538827,2150


In [95]:
# Verificar duplicados en coordenadas
if coordenadas.duplicated(subset=['RegionID']).any():
    print("Aún hay duplicados en la tabla coordenadas.")
else:
    print("No hay duplicados en la tabla coordenadas.")

No hay duplicados en la tabla coordenadas.
